In [ ]:
# Simulation imports
from reporting import simulation
from reporting.plot_outbreak import plot_outbreak

In [ ]:
# Library imports
import geopandas
import matplotlib
import shapely
import numpy as np
from matplotlib import pyplot

# Set default plot size
matplotlib.rcParams['figure.figsize'] = [16.0, 12.0]




# Day 1
## All the field reports have come in and it's time to analyse them

In [ ]:
simulation.gather_reports_for_day_1()

### Let's grab the data directly from the database

In [ ]:
from psycopg2 import connect
connection = connect(dbname='johnsnow')
cases = geopandas.read_postgis(
    "select * from reporting_report", connection,
    geom_col='location',
    crs={'init': 'epsg:4326'}  # The most widely used spatial reference system
)


- `cases` is a Geopandas `GeoDataFrame`
- This is just like a regular `DataFrame` except that it has to have a `GeoSeries` column containing spatial data
- we have lots of interesting new spatial methods that will act on this geometry column


In [ ]:
cases[['doctor_name', 'patient_name', 'diagnosis', 'location']].sample(5)

### Let's visualise

In [ ]:
cases.plot(marker="o", color="red", markersize=64, alpha=0.2)

### This looks scary
- We have obvious clusters of diagnoses

### Let's distinguish different diagnoses

In [ ]:
plot = cases.plot(marker="o", markersize=64, alpha=0.5, column='diagnosis')

### This situation doesn't look right
- Each cluster has many different diseases diagnosed within it.

### This data is useless without context. Let's map it
- We will import "Shapefiles" from the Open Street Map project
- Shapefiles are a very common format for sharing spatial data along with facts about objects
- You can say:
  - This is a road
  - It has a shape
  - It has a name: "Oxford Street"
  - It's classified as a B-Road

In [ ]:
# Import London's roads
roads = geopandas.read_file('../open-street-map-data/london-roads_shp/')
roads.plot(color="black", alpha=0.2)

In [ ]:
# Filter roads by type
mains = roads[roads['highway'].isin(('trunk', 'primary', 'secondary', 'tertiary', 'unclassified'))]
mains.plot(color="black", alpha=0.2)

In [ ]:
# Import London's rivers
rivers = geopandas.read_file('../open-street-map-data/london-rivers_shp/')

Let's use that extra data to filter the shapes we've imported:

In [ ]:
# Draw all three datasets on one axis to make a map
figure, axes = pyplot.subplots()

rivers.plot(ax=axes, color='blue', alpha=0.3)
mains.plot(ax=axes, color="black", alpha=0.15)
cases.plot(ax=axes, marker="o", column="diagnosis", markersize=128, alpha=0.5)

pyplot.show()

### What's going on? I have a hunch...

In [ ]:
# Let's get open street map data about medical facilities
medical = geopandas.read_file('../open-street-map-data/london-hospitals_shp/')

# And use pandas filtering to select only the hospitals
hospitals = medical[medical['amenity'] == 'hospital'].copy()

In [ ]:
# Overlay our various geo-datasets
figure, axes = pyplot.subplots()
rivers_plot = rivers.plot(ax=axes, color='blue', alpha=0.3)
map_plot = mains.plot(ax=axes, color="black", alpha=0.2)
cases_plot = cases.plot(ax=axes, marker="o", column="diagnosis", markersize=64, alpha=0.5)

# Plot a green cross at the location of each hospital
final = hospitals.plot(ax=axes, marker="P", markersize=1000, color="green", alpha=0.4)

pyplot.show()

### We need to improve our analysis

In [ ]:
# Create a polygon representing about 150m around each hospital
hospitals['geometry'] = hospitals['geometry'].buffer(0.0015)
hospitals.plot()

![Joins](spatial-join.png)

In [ ]:
# Join together the hospital zones with the cases
# The "join" is a spatial one - we are joining cases with the hospitals that they are close to
joined_cases = geopandas.sjoin(
    left_df=cases,
    right_df=hospitals,
    how='left',
    op='within',
)

In [ ]:
# Now each case that happened in a hospital is associated with the relevant hospital
joined_cases[['doctor_name', 'patient_name', 'diagnosis', 'name']].sample(10)

### Let's find cases that didn't happen at hospitals

In [ ]:
hospital_cases = joined_cases[~joined_cases['name'].isnull()]
non_hospital_cases = joined_cases[joined_cases['name'].isnull()]

axes = hospital_cases.plot(color="green")
non_hospital_cases.plot(ax=axes, color="red")


### And let's pull everything together

In [ ]:
plot_outbreak()

# Day 2
### Not so peaceful

In [ ]:
simulation.gather_reports_for_day_2()
plot_outbreak()

### If I was John Snow, I would investigate...

# Day 3

In [ ]:
simulation.gather_reports_for_day_3()
plot_outbreak()

# 28 days later

In [ ]:
simulation.gather_reports_for_day_28()
plot_outbreak()